### Chapter 22: Debugging and Best Practices

#### Common Coding Mistakes

##### Clock Domain Crossing (CDC) Issues

**Problem:** Improper handling of signals crossing clock domains can cause metastability.

```systemverilog
// BAD: Direct signal crossing without synchronization
module bad_cdc (
    input clk_a, clk_b, rst_n,
    input data_a,
    output logic data_b
);
    always_ff @(posedge clk_b or negedge rst_n) begin
        if (!rst_n) data_b <= 1'b0;
        else data_b <= data_a;  // Metastability risk!
    end
endmodule

// GOOD: Proper synchronization
module good_cdc (
    input clk_a, clk_b, rst_n,
    input data_a,
    output logic data_b
);
    logic sync_ff1, sync_ff2;
    
    always_ff @(posedge clk_b or negedge rst_n) begin
        if (!rst_n) begin
            sync_ff1 <= 1'b0;
            sync_ff2 <= 1'b0;
        end else begin
            sync_ff1 <= data_a;
            sync_ff2 <= sync_ff1;
        end
    end
    
    assign data_b = sync_ff2;
endmodule
```

##### Blocking vs Non-blocking Assignments

**Problem:** Mixing blocking and non-blocking assignments incorrectly.

```systemverilog
// BAD: Race conditions and simulation/synthesis mismatch
always_ff @(posedge clk) begin
    a = b;      // Blocking - executes immediately
    c <= a;     // Non-blocking - scheduled for end of time step
end

// GOOD: Consistent use of non-blocking for sequential logic
always_ff @(posedge clk) begin
    a <= b;     // All non-blocking
    c <= a;     // Uses previous value of 'a'
end

// GOOD: Blocking for combinational logic
always_comb begin
    temp = a & b;
    result = temp | c;
end
```

##### Incomplete Sensitivity Lists

**Problem:** Missing signals in sensitivity lists can cause simulation issues.

```systemverilog
// BAD: Incomplete sensitivity list
always @(a) begin  // Missing 'b' in sensitivity list
    if (sel)
        out = a;
    else
        out = b;   // Changes to 'b' won't trigger the block
end

// GOOD: Complete sensitivity list or use always_comb
always_comb begin
    if (sel)
        out = a;
    else
        out = b;
end
```

##### Latch Inference

**Problem:** Unintentional latch creation due to incomplete case statements.

```systemverilog
// BAD: Creates latches
always_comb begin
    case (sel)
        2'b00: out = a;
        2'b01: out = b;
        // Missing cases cause latch inference
    endcase
end

// GOOD: Complete case statement
always_comb begin
    case (sel)
        2'b00: out = a;
        2'b01: out = b;
        2'b10: out = c;
        default: out = d;  // Always include default
    endcase
end
```

##### Reset Strategy Issues

**Problem:** Inconsistent or incomplete reset handling.

```systemverilog
// BAD: Incomplete reset
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        counter <= 0;
        // Missing reset for 'flag' - creates mixed reset domains
    end else begin
        counter <= counter + 1;
        flag <= (counter == 10);
    end
end

// GOOD: Complete reset
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        counter <= 0;
        flag <= 1'b0;
    end else begin
        counter <= counter + 1;
        flag <= (counter == 10);
    end
end
```


#### Debugging Techniques

##### Assertion-Based Verification

Use SystemVerilog Assertions (SVA) to catch design errors early.

```systemverilog
module fifo_with_assertions #(
    parameter DEPTH = 8,
    parameter WIDTH = 32
)(
    input clk, rst_n,
    input push, pop,
    input [WIDTH-1:0] data_in,
    output logic [WIDTH-1:0] data_out,
    output logic full, empty
);
    
    logic [$clog2(DEPTH):0] count;
    
    // Functional assertions
    assert property (@(posedge clk) disable iff (!rst_n)
        push && full |-> !push)
        else $error("Push attempted when FIFO is full");
    
    assert property (@(posedge clk) disable iff (!rst_n)
        pop && empty |-> !pop)
        else $error("Pop attempted when FIFO is empty");
    
    // Cover properties to ensure scenarios are tested
    cover property (@(posedge clk) disable iff (!rst_n)
        $rose(full));
    
    cover property (@(posedge clk) disable iff (!rst_n)
        $rose(empty));
        
endmodule
```

##### Debug Probes and Monitors

Create reusable debug components for complex designs.

```systemverilog
// Debug monitor for AXI transactions
module axi_debug_monitor #(
    parameter ADDR_WIDTH = 32,
    parameter DATA_WIDTH = 32
)(
    input clk, rst_n,
    // AXI interface
    input [ADDR_WIDTH-1:0] awaddr,
    input awvalid, awready,
    input [DATA_WIDTH-1:0] wdata,
    input wvalid, wready,
    input bvalid, bready
);
    
    // Transaction tracking
    int write_count = 0;
    int outstanding_writes = 0;
    
    // Monitor write address channel
    always @(posedge clk) begin
        if (awvalid && awready) begin
            $display("Time %0t: Write Address: 0x%h", $time, awaddr);
            outstanding_writes++;
        end
    end
    
    // Monitor write data channel
    always @(posedge clk) begin
        if (wvalid && wready) begin
            $display("Time %0t: Write Data: 0x%h", $time, wdata);
            write_count++;
        end
    end
    
    // Monitor write response
    always @(posedge clk) begin
        if (bvalid && bready) begin
            outstanding_writes--;
            $display("Time %0t: Write Complete, Outstanding: %0d", 
                    $time, outstanding_writes);
        end
    end
    
    // Deadlock detection
    always @(posedge clk) begin
        if (outstanding_writes > 0) begin
            static int timeout_counter = 0;
            timeout_counter++;
            if (timeout_counter > 1000) begin
                $error("Potential deadlock detected: %0d outstanding writes", 
                       outstanding_writes);
            end
        end else begin
            timeout_counter = 0;
        end
    end
    
endmodule
```

##### Waveform Analysis Techniques

```systemverilog
// Testbench with comprehensive waveform dumping
module tb_processor;
    
    // DUT signals
    logic clk, rst_n;
    logic [31:0] instruction, pc;
    
    processor dut (.*);
    
    initial begin
        // Configure waveform dumping
        $dumpfile("processor_waves.vcd");
        $dumpvars(0, tb_processor);
        
        // Add specific signal groups for easier analysis
        $dumpvars(1, dut.cpu_core);
        $dumpvars(1, dut.cache);
        $dumpvars(1, dut.memory_controller);
        
        // Test sequence
        reset_sequence();
        run_test_program();
        
        #1000 $finish;
    end
    
    // Generate clock
    always #5 clk = ~clk;
    
    // Reset sequence
    task reset_sequence();
        rst_n = 0;
        repeat(10) @(posedge clk);
        rst_n = 1;
        $display("Reset complete at time %0t", $time);
    endtask
    
endmodule
```

##### Debugging with $display and $monitor

```systemverilog
module debug_example;
    
    // Use hierarchical $display for complex designs
    task debug_transaction(string phase, int id, logic [31:0] addr);
        $display("[%0t] %s: Transaction %0d, Address: 0x%h", 
                $time, phase, id, addr);
    endtask
    
    // Conditional debugging
    parameter DEBUG_LEVEL = 2;
    
    task debug_print(int level, string msg);
        if (level <= DEBUG_LEVEL) begin
            $display("[DEBUG L%0d] %s", level, msg);
        end
    endtask
    
    // State machine debugging
    typedef enum {IDLE, ACTIVE, WAIT, DONE} state_t;
    state_t current_state, next_state;
    
    always @(posedge clk) begin
        if (current_state != next_state) begin
            $display("State transition: %s -> %s", 
                    current_state.name(), next_state.name());
        end
    end
    
endmodule
```

#### Simulation and Synthesis Considerations

##### Simulation vs Synthesis Differences

Understanding constructs that behave differently in simulation and synthesis.

```systemverilog
// Timing-sensitive code - simulation only
module timing_sensitive_bad;
    logic clk, data, output_reg;
    
    // BAD: Timing-dependent code
    always @(posedge clk) begin
        output_reg <= data;
        #1 data <= ~data;  // Delay - simulation only
    end
endmodule

// Synthesizable equivalent
module timing_proper;
    logic clk, data, output_reg;
    logic data_delayed;
    
    always_ff @(posedge clk) begin
        output_reg <= data;
        data_delayed <= ~data;
    end
    
    assign data = data_delayed;
endmodule
```

##### Synthesis Attributes and Pragmas

```systemverilog
module synthesis_attributes;
    
    // Synthesis attributes for optimization
    (* keep *) logic important_signal;
    (* dont_touch *) logic debug_signal;
    
    // RAM inference with attributes
    (* ram_style = "block" *) logic [31:0] memory [0:1023];
    
    // Clock domain crossing attribute
    (* async_reg = "true" *) logic sync_ff1, sync_ff2;
    
    // FSM encoding
    (* fsm_encoding = "one_hot" *)
    typedef enum logic [3:0] {
        IDLE   = 4'b0001,
        ACTIVE = 4'b0010,
        WAIT   = 4'b0100,
        DONE   = 4'b1000
    } state_t;
    
endmodule
```

##### Simulation-Only Constructs

```systemverilog
module simulation_constructs;
    
    `ifdef SIMULATION
        // Simulation-only code
        initial begin
            $timeformat(-9, 2, " ns", 10);
            $display("Simulation started");
        end
        
        // Assertion for simulation
        assert property (@(posedge clk) req |-> ##[1:3] ack)
            else $error("Handshake protocol violation");
            
    `endif
    
    // Synthesis pragma
    // synthesis translate_off
    always @(posedge clk) begin
        if ($time > 10000) begin
            $display("Long simulation detected");
        end
    end
    // synthesis translate_on
    
endmodule
```

#### Coding Style Guidelines

##### Naming Conventions

```systemverilog
// Consistent naming conventions
module memory_controller (
    // Clock and reset (lowercase with underscores)
    input  logic        clk,
    input  logic        rst_n,
    
    // Interface signals (descriptive names)
    input  logic        read_enable,
    input  logic        write_enable,
    input  logic [31:0] address,
    input  logic [31:0] write_data,
    output logic [31:0] read_data,
    output logic        ready,
    
    // Bus interface (prefix for grouping)
    output logic        mem_req,
    output logic [31:0] mem_addr,
    input  logic        mem_ack
);

    // Internal signals (clear, descriptive names)
    logic        internal_busy;
    logic [31:0] address_register;
    logic [31:0] data_buffer;
    
    // State machine (enumerated type with meaningful names)
    typedef enum logic [1:0] {
        IDLE_STATE,
        READ_STATE,
        WRITE_STATE,
        WAIT_STATE
    } controller_state_t;
    
    controller_state_t current_state, next_state;
    
endmodule
```

##### Code Organization and Structure

```systemverilog
// Well-structured module template
module template_module #(
    // Parameters first, with default values
    parameter int WIDTH = 32,
    parameter int DEPTH = 1024,
    parameter bit ENABLE_DEBUG = 1'b0
)(
    // Port list organized by function
    // Clock and reset first
    input  logic                    clk,
    input  logic                    rst_n,
    
    // Control signals
    input  logic                    enable,
    input  logic                    start,
    output logic                    done,
    output logic                    error,
    
    // Data interface
    input  logic [WIDTH-1:0]        data_in,
    output logic [WIDTH-1:0]        data_out,
    input  logic                    valid_in,
    output logic                    valid_out,
    
    // Memory interface
    output logic [$clog2(DEPTH)-1:0] mem_addr,
    output logic [WIDTH-1:0]        mem_data,
    output logic                    mem_we
);

    // Local parameters
    localparam int ADDR_WIDTH = $clog2(DEPTH);
    
    // Type definitions
    typedef struct packed {
        logic [WIDTH-1:0] data;
        logic             valid;
        logic             error;
    } data_packet_t;
    
    // Internal signal declarations (grouped by function)
    // Control signals
    logic controller_enable;
    logic operation_complete;
    
    // Data path signals
    data_packet_t input_packet, output_packet;
    logic [WIDTH-1:0] processed_data;
    
    // Memory interface signals
    logic [ADDR_WIDTH-1:0] memory_address;
    logic memory_write_enable;
    
    // Continuous assignments
    assign data_out = output_packet.data;
    assign valid_out = output_packet.valid;
    
    // Sequential logic blocks
    always_ff @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            // Reset logic
        end else begin
            // Main sequential logic
        end
    end
    
    // Combinational logic blocks
    always_comb begin
        // Combinational logic
    end
    
    // Generate blocks for parameterized structures
    generate
        if (ENABLE_DEBUG) begin : gen_debug
            // Debug logic
        end
    endgenerate
    
endmodule
```

##### Documentation Standards

```systemverilog
/**
 * Memory Controller Module
 * 
 * Description:
 *   This module implements a memory controller with configurable width
 *   and depth. It provides a simple read/write interface with handshaking.
 * 
 * Parameters:
 *   WIDTH       - Data width in bits (default: 32)
 *   DEPTH       - Memory depth in words (default: 1024)
 *   ENABLE_ECC  - Enable error correction (default: 0)
 * 
 * Interfaces:
 *   - CPU Interface: Simple read/write with ready/valid handshaking
 *   - Memory Interface: Standard memory interface with address/data/control
 * 
 * Author: Design Team
 * Date: 2024-01-15
 * Version: 1.2
 */
module memory_controller #(
    parameter int WIDTH = 32,        ///< Data bus width
    parameter int DEPTH = 1024,      ///< Memory depth in words
    parameter bit ENABLE_ECC = 1'b0  ///< Enable ECC protection
)(
    input  logic clk,                ///< System clock
    input  logic rst_n,              ///< Active-low async reset
    
    // CPU Interface
    input  logic        cpu_req,     ///< CPU request signal
    input  logic        cpu_we,      ///< CPU write enable (1=write, 0=read)
    input  logic [31:0] cpu_addr,    ///< CPU address
    input  logic [31:0] cpu_wdata,   ///< CPU write data
    output logic [31:0] cpu_rdata,   ///< CPU read data
    output logic        cpu_ready    ///< CPU ready signal
);

    // Implementation details...
    
endmodule
```

#### Performance Optimization

##### Pipeline Design Techniques

```systemverilog
// Efficient pipeline design
module optimized_pipeline #(
    parameter int STAGES = 4,
    parameter int WIDTH = 32
)(
    input  logic clk, rst_n,
    input  logic [WIDTH-1:0] data_in,
    input  logic valid_in,
    output logic [WIDTH-1:0] data_out,
    output logic valid_out
);

    // Pipeline registers
    logic [WIDTH-1:0] stage_data [STAGES-1:0];
    logic             stage_valid [STAGES-1:0];
    
    // Optimized pipeline with generate
    generate
        for (genvar i = 0; i < STAGES; i++) begin : gen_stages
            always_ff @(posedge clk or negedge rst_n) begin
                if (!rst_n) begin
                    stage_data[i] <= '0;
                    stage_valid[i] <= 1'b0;
                end else begin
                    if (i == 0) begin
                        stage_data[i] <= data_in;
                        stage_valid[i] <= valid_in;
                    end else begin
                        stage_data[i] <= stage_data[i-1];
                        stage_valid[i] <= stage_valid[i-1];
                    end
                end
            end
            
            // Processing logic for each stage
            if (i == 1) begin : stage1_process
                // Stage 1 specific processing
            end else if (i == 2) begin : stage2_process
                // Stage 2 specific processing
            end
        end
    endgenerate
    
    assign data_out = stage_data[STAGES-1];
    assign valid_out = stage_valid[STAGES-1];
    
endmodule
```

##### Resource Optimization

```systemverilog
// Efficient resource usage
module resource_optimized #(
    parameter int DATA_WIDTH = 32,
    parameter int FIFO_DEPTH = 16
)(
    input logic clk, rst_n,
    input logic push, pop,
    input logic [DATA_WIDTH-1:0] data_in,
    output logic [DATA_WIDTH-1:0] data_out,
    output logic full, empty
);

    // Use power-of-2 depth for efficient address generation
    localparam int ADDR_WIDTH = $clog2(FIFO_DEPTH);
    
    // Memory array
    logic [DATA_WIDTH-1:0] memory [FIFO_DEPTH-1:0];
    
    // Efficient pointer management
    logic [ADDR_WIDTH:0] write_ptr, read_ptr;  // Extra bit for full/empty detection
    
    // Optimized full/empty detection
    assign full = (write_ptr == {~read_ptr[ADDR_WIDTH], read_ptr[ADDR_WIDTH-1:0]});
    assign empty = (write_ptr == read_ptr);
    
    // Memory operations
    always_ff @(posedge clk) begin
        if (push && !full) begin
            memory[write_ptr[ADDR_WIDTH-1:0]] <= data_in;
        end
    end
    
    always_ff @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            write_ptr <= '0;
            read_ptr <= '0;
        end else begin
            if (push && !full) write_ptr <= write_ptr + 1;
            if (pop && !empty) read_ptr <= read_ptr + 1;
        end
    end
    
    assign data_out = memory[read_ptr[ADDR_WIDTH-1:0]];
    
endmodule
```

##### Timing Optimization

```systemverilog
// Timing-optimized design
module timing_optimized (
    input logic clk, rst_n,
    input logic [31:0] a, b, c, d,
    output logic [31:0] result
);

    // Pipeline complex operations
    logic [31:0] stage1_result, stage2_result;
    
    // Stage 1: Parallel operations
    always_ff @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            stage1_result <= '0;
        end else begin
            stage1_result <= a + b;  // First addition
        end
    end
    
    // Stage 2: Use previous result
    always_ff @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            stage2_result <= '0;
        end else begin
            stage2_result <= stage1_result + (c * d);  // Multiply in parallel
        end
    end
    
    assign result = stage2_result;
    
endmodule

// Alternative: Fully combinational with careful timing
module timing_balanced (
    input logic clk, rst_n,
    input logic [15:0] a, b, c, d,  // Reduced width for timing
    output logic [31:0] result
);

    // Break critical path with intermediate signals
    logic [31:0] mult_result, add_result;
    
    always_comb begin
        mult_result = c * d;        // Parallel multiply
        add_result = a + b;         // Parallel add
        result = mult_result + add_result;  // Final add
    end
    
endmodule
```

##### Power Optimization

```systemverilog
// Power-optimized design
module power_optimized #(
    parameter int WIDTH = 32
)(
    input logic clk, rst_n,
    input logic enable,
    input logic [WIDTH-1:0] data_in,
    output logic [WIDTH-1:0] data_out
);

    // Clock gating for power savings
    logic gated_clk;
    
    // Clock gate (use vendor-specific cells in real designs)
    assign gated_clk = clk & enable;
    
    // Registers with clock gating
    logic [WIDTH-1:0] data_reg;
    
    always_ff @(posedge gated_clk or negedge rst_n) begin
        if (!rst_n) begin
            data_reg <= '0;
        end else begin
            data_reg <= data_in;
        end
    end
    
    // Power-aware multiplexing
    always_comb begin
        if (enable) begin
            data_out = data_reg;
        end else begin
            data_out = '0;  // Drive to known state to reduce switching
        end
    end
    
endmodule
```

#### Best Practices Summary

1. **Design for Testability**: Include debug hooks and assertion monitors
2. **Use Consistent Coding Style**: Follow naming conventions and code organization
3. **Optimize for Target Technology**: Consider FPGA vs ASIC differences
4. **Plan for Reusability**: Parameterize designs and use proper interfaces
5. **Document Thoroughly**: Include clear comments and interface descriptions
6. **Verify Early and Often**: Use assertions and comprehensive testbenches
7. **Consider Power and Timing**: Design with constraints in mind from the start
8. **Use Modern SystemVerilog Features**: Interfaces, packages, and proper data types

These practices will help you write robust, maintainable, and efficient SystemVerilog code that works reliably in both simulation and synthesis environments.